In [1]:
!pip install requests


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# only use this if your packages are installing to the wrong location
import sys
sys.path.append("c:/users/beau/anaconda3/lib/site-packages")

In [3]:
import os
import time
import requests
import yaml
import pandas as pd
import numpy as np

In [4]:
with open('../../config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
season_year = 2024

In [34]:
config['basketball']['ROOT']

'C:/Users/Beau/Documents/GitHub/fantasy-basketball'

### Sportradar for stats

In [8]:
base_sportradar_url = "https://api.sportradar.com/nba/"
access_level = 'trial'
language_code = 'en'
season_type = 'REG'
headers = {
    'x-api-key': config['sportradar_api_key'],
}

In [31]:
def get_teams():
    query = f"{base_sportradar_url}/{access_level}/v7/{language_code}/league/teams.json"
    try:
        response = requests.get(query, headers=headers)
        response.raise_for_status()
        teams = response.json()
    except requests.exceptions.RequestException as error:
        print(f"An error occurred: {error}")

    df_teams = pd.DataFrame(teams['teams'])
    df_teams['NBA'] = df_teams.id.str.contains('583')
    df_nba_teams = df_teams.loc[df_teams.NBA]
    
    return df_nba_teams

In [51]:
# df_teams = get_teams()
# df_nba_teams.to_csv(os.path.join(config['basketball']['ROOT'], "data", "teams.csv"), index=False)
df_nba_teams = pd.read_csv(os.path.join(config['basketball']['ROOT'], "data", "teams.csv"))

In [52]:
df_nba_teams.head()

,id,name,alias,market,NBA
0,583ec87d-fb46-11e1-82cb-f4ce4684ea4c,76ers,PHI,Philadelphia,True
1,583ecefd-fb46-11e1-82cb-f4ce4684ea4c,Bucks,MIL,Milwaukee,True
2,583ec5fd-fb46-11e1-82cb-f4ce4684ea4c,Bulls,CHI,Chicago,True
3,583ec773-fb46-11e1-82cb-f4ce4684ea4c,Cavaliers,CLE,Cleveland,True
4,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,Celtics,BOS,Boston,True


In [53]:
team_id = '583ec87d-fb46-11e1-82cb-f4ce4684ea4c'

In [54]:
query = f"{base_sportradar_url}/{access_level}/v8/{language_code}/seasons/{season_year}/{season_type}/teams/{team_id}/statistics.json"
try:
    response = requests.get(query, headers=headers)
    response.raise_for_status()
    data = response.json()
except requests.exceptions.RequestException as error:
    print(f"An error occurred: {error}")

In [56]:
data.keys()

dict_keys(['id', 'name', 'market', 'sr_id', 'reference', 'season', 'own_record', 'opponents', 'players'])

In [72]:
pd.DataFrame(data['players']).loc[0]['average']

{'minutes': 4.3,
 'points': 2.3,
 'off_rebounds': 0.75,
 'def_rebounds': 0.25,
 'rebounds': 1.0,
 'assists': 0.3,
 'steals': 0.0,
 'blocks': 0.0,
 'turnovers': 1.0,
 'personal_fouls': 0.0,
 'flagrant_fouls': 0.0,
 'blocked_att': 0.0,
 'field_goals_made': 1.0,
 'field_goals_att': 2.0,
 'three_points_made': 0.0,
 'three_points_att': 0.75,
 'free_throws_made': 0.25,
 'free_throws_att': 0.25,
 'two_points_made': 1.0,
 'two_points_att': 1.25,
 'efficiency': 1.5,
 'true_shooting_att': 2.11,
 'points_off_turnovers': 1.0,
 'points_in_paint_made': 1.0,
 'points_in_paint_att': 1.0,
 'points_in_paint': 2.0,
 'fouls_drawn': 0.25,
 'offensive_fouls': 0.0,
 'fast_break_pts': 0.0,
 'fast_break_att': 0.25,
 'fast_break_made': 0.0,
 'second_chance_pts': 1.25,
 'second_chance_att': 0.75,
 'second_chance_made': 0.5}

In [6]:
def get_games_in_season(season_year):
    query = f"{base_sportradar_url}/{access_level}/v8/{language_code}/games/{season_year}/{season_type}/schedule.json"
    try:
        response = requests.get(query, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as error:
        print(f"An error occurred: {error}")

    df_season_weeks = pd.DataFrame(data['weeks'])

    games_list = []
    for games in df_season_weeks.games:
        games_list.append(pd.DataFrame(games))

    all_games = pd.concat(games_list).reset_index(drop=True)

    all_games['home_team_id'] = all_games.home.apply(lambda x: x['id'])
    all_games['away_team_id'] = all_games.away.apply(lambda x: x['id'])
    
    return all_games

In [ ]:
season_year = 2024
schedule = get_games_in_season(season_year)
schedule.to_csv(os.path.join(config['ROOT'], "data", f"schedule_{season_year}.csv"), index=False)
# schedule = pd.read_csv(os.path.join(config['ROOT'], "data", f"{season_year}", f"schedule.csv"))